# Streaming final agent output | 最終エージェントの出力をストリーミング



エージェントの最終出力のみをストリーミングしたい場合は、`FinalStreamingStdOutCallbackHandler` コールバックを使用できます。これを使用するには、基盤となるLLMもストリーミングをサポートしている必要があります。

> If you only want the final output of an agent to be streamed, you can use the callback `FinalStreamingStdOutCallbackHandler`.
> For this, the underlying LLM has to support streaming as well.



In [1]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
from langchain.llms import OpenAI

基礎となるLLMを`streaming = True`で作成し、`FinalStreamingStdOutCallbackHandler`の新しいインスタンスを渡しましょう。

> Let's create the underlying LLM with `streaming = True` and pass a new instance of `FinalStreamingStdOutCallbackHandler`.



In [2]:
llm = OpenAI(
    streaming=True, callbacks=[FinalStreamingStdOutCallbackHandler()], temperature=0
)

In [4]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)
agent.run(
    "It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany."
)

 Konrad Adenauer became Chancellor of Germany in 1949, 74 years ago in 2023.

'Konrad Adenauer became Chancellor of Germany in 1949, 74 years ago in 2023.'

### Handling custom answer prefixes | カスタム回答プレフィックスの処理



デフォルトでは、トークンのシーケンス「Final」「Answer」「:」が、エージェントが答えを出したことを示していると解釈されます。しかし、答えの接頭辞として使用するカスタムシーケンスを指定することもできます。

> By default, we assume that the token sequence `"Final", "Answer", ":"` indicates that the agent has reached an answers. We can, however, also pass a custom sequence to use as answer prefix.



In [5]:
llm = OpenAI(
    streaming=True,
    callbacks=[
        FinalStreamingStdOutCallbackHandler(answer_prefix_tokens=["The", "answer", ":"])
    ],
    temperature=0,
)

利便性のために、コールバックは`answer_prefix_tokens`と比較する際に自動的に空白や改行文字を除去します。例えば、`answer_prefix_tokens = ["The", " answer", ":"]`であれば、`["\nThe", " answer", ":"]`も`["The", " answer", ":"]`も答えの接頭辞として認識されることになります。

> For convenience, the callback automatically strips whitespaces and new line characters when comparing to `answer_prefix_tokens`. I.e., if `answer_prefix_tokens = ["The", " answer", ":"]` then both `["\nThe", " answer", ":"]` and `["The", " answer", ":"]` would be recognized a the answer prefix.



もし回答のプレフィックスのトークン化されたバージョンがわからない場合は、以下のコードでそれを決定することができます：

> If you don't know the tokenized version of your answer prefix, you can determine it with the following code:



In [ ]:
from langchain.callbacks.base import BaseCallbackHandler


class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs) -> None:
        # print every token on a new line
        print(f"#{token}#")


llm = OpenAI(streaming=True, callbacks=[MyCallbackHandler()])
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)
agent.run(
    "It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany."
)

### Also streaming the answer prefixes | 回答のプレフィックスも同時にストリーミングされます



`stream_prefix = True` パラメータが設定されている場合、回答のプレフィックス自体もストリーミングされます。これは、回答のプレフィックス自体が回答の一部となる場合に便利です。例えば、あなたの回答が JSON のような形式である場合に該当します

> When the parameter `stream_prefix = True` is set, the answer prefix itself will also be streamed. This can be useful when the answer prefix itself is part of the answer. For example, when your answer is a JSON like

{ "action": "最終回答", "action\_input": "コンラート・アデナウアーは74年前に首相に就任しました。" }

> `{
>     "action": "Final answer",
>     "action_input": "Konrad Adenauer became Chancellor 74 years ago."
> }`

そして、`action_input`だけでなく、JSON全体がストリーミングされることを望んでいます。

> and you don't only want the `action_input` to be streamed, but the entire JSON.

